In [1]:
# IFC Parser
# Based on:
# https://thinkmoult.com/using-ifcopenshell-parse-ifc-files-python.html
# https://github.com/johannesmichael/ifc-python/blob/master/modules/ifc_pset_utils.py
# https://www.kaggle.com/code/ponybiam/introduction-to-ifcopenshell-functions/notebook
# https://blenderbim.org/docs-python/ifcopenshell-python/hello_world.html

import ifcopenshell
import ifcHelper 
import pandas as pd
import numpy as np

## Import the file

In [2]:
try:
  ifc_file = ifcopenshell.open('ifc_test_file.ifc')
  print("File loaded successfully")
except:
  print("An exception occurred")



File loaded successfully


In [4]:
print ("File version:")
print (ifc_file.schema)
## Script is written for IFC2X3

File version:
IFC2X3


In [5]:
no_walls = len(ifc_file.by_type("IfcWall"))
no_windows = len(ifc_file.by_type("IfcWindow"))
no_doors = len(ifc_file.by_type("IfcDoor"))
no_slabs = len(ifc_file.by_type("IfcSlab"))
no_flo_segments = len(ifc_file.by_type("IfcFlowSegment"))

print("There are {} walls, {} windows, {} doors , {} slabs and {} flow segments in the file".format(no_walls,no_windows,no_doors, no_slabs, no_flo_segments))

There are 10 walls, 2 windows, 1 doors , 5 slabs and 0 flow segments in the file


## Get the info about Walls

In [6]:
## Get all walls by_type: IfcWall
##
walls = ifc_file.by_type('IfcWall')

In [7]:
walls[0]

#183=IfcWallStandardCase('0XKS9_JrzCLBvqx8CYyH2y',#41,'Basic Wall:Z01:331325',$,'Basic Wall:Z01',#137,#179,'331325')

In [8]:
# print "a_wall"s Name attribute
print("The Name of walls[0] is %s"%(walls[0].ObjectType))
# print "a_wall"s Description attribute
print("The Description of walls[0] is %s"%(walls[0].Description))

The Name of walls[0] is Basic Wall:Z01
The Description of walls[0] is None


In [9]:
## Wall elevation area can be calculated using the volume/thickness

In [10]:
walls_volumes = list()
walls_thickness = list()
walls_areas = list()
walls_types = list()
for i in range(len(walls)):
    quantities = ifcHelper.get_quantity_single_value(ifcHelper.get_related_quantities(walls[i])[0])
    walls_volumes.append(quantities['NetVolume'])
    walls_thickness.append(quantities['Width']/1000)
    walls_areas.append(walls_volumes[i]/walls_thickness[i])
    walls_types.append(walls[i].ObjectType)
    
## Get volumes, thickness and calculate areas
## Get wall types

In [11]:
print ('Total wall area in the IFC file:')
print (sum(walls_areas))

Total wall area in the IFC file:
552.9350204145787


In [12]:
d = {'Type':walls_types,'Area':walls_areas}
wall_df = pd.DataFrame(d)
wall_types_sum = wall_df.groupby('Type')['Area'].sum()

In [13]:
 wall_types_sum_df = pd.DataFrame(wall_types_sum)

In [14]:
 wall_types_sum_df 

,Area
Type,
Basic Wall:Z01,323.497947
Basic Wall:Z02,229.437074


## Get the info about Slabs


In [15]:
slabs = ifc_file.by_type('IfcSlab')

In [16]:

slabs_areas = list()
slabs_types = list()
for i in range(len(slabs)):
    quantities_slabs = ifcHelper.get_quantity_single_value(ifcHelper.get_related_quantities(slabs[i])[0])
    slabs_areas.append(quantities_slabs['NetArea'])
   

    slabs_types.append(slabs[i].ObjectType)
    
## Get volumes, thickness and calculate areas
## Get wall types

In [17]:
d_s = {'Type':slabs_types,'Area':slabs_areas}
slabs_df = pd.DataFrame(d_s)
slabs_types_sum = slabs_df.groupby('Type')['Area'].sum()

In [18]:
 slabs_types_sum_df = pd.DataFrame(slabs_types_sum)

In [19]:
 slabs_types_sum_df

,Area
Type,
Basic Roof:Ogólne - 400 mm,1992.181745
Floor:Ogólne 150 mm,377.670000


## Get the info about Windows


In [20]:
windows = ifc_file.by_type('IfcWindow')

In [21]:
windows

[#1557=IfcWindow('0XKS9_JrzCLBvqv88YyH1n',#41,'M_Stałe:O02:331504',$,'M_Stałe:O02',#15459,#1549,'331504',1830.,915.),
 #1787=IfcWindow('0XKS9_JrzCLBvqv88YyH1s',#41,'M_Stałe:O01:331511',$,'M_Stałe:O01',#15494,#1779,'331511',609.999999999999,406.)]

In [22]:

print("The Name of windows[0] is %s"%(windows[0].ObjectType))

print("The Description of windows[0] is %s"%(windows[0].Description))

The Name of windows[0] is M_Stałe:O02
The Description of windows[0] is None


In [23]:
quantities = ifcHelper.get_quantity_single_value(ifcHelper.get_related_quantities(windows[0])[0])

In [24]:
quantities

{'Height': 1830.0, 'Width': 915.0, 'Area': 3.17875400000013}

In [25]:
windows_types = list()
windows_areas = list()
for i in range(len(windows)):
    quantities_windows = ifcHelper.get_quantity_single_value(ifcHelper.get_related_quantities(windows[i])[0])
    windows_areas.append(quantities_windows['Area'])
    windows_types.append(windows[i].ObjectType)

In [26]:
w_s = {'Type':windows_types,'Area':windows_areas}
windows_df = pd.DataFrame(w_s)
windows_types_sum = windows_df.groupby('Type')['Area'].sum()

In [27]:
windows_types_sum_df =  pd.DataFrame(windows_types_sum)

In [28]:
windows_types_sum_df

,Area
Type,
M_Stałe:O01,0.776808
M_Stałe:O02,3.178754


## Get the info about Doors


In [29]:
doors = ifc_file.by_type('IfcDoor')

In [30]:
quantities = ifcHelper.get_quantity_single_value(ifcHelper.get_related_quantities(doors[0])[0])

In [31]:
quantities

{'Height': 2134.0, 'Width': 915.0, 'Area': 3.18957899999998}

In [32]:
doors_types = list()
doors_areas = list()
for i in range(len(doors)):
    quantities_doors = ifcHelper.get_quantity_single_value(ifcHelper.get_related_quantities(doors[i])[0])
    doors_areas.append(quantities_doors['Area'])
    doors_types.append(doors[i].ObjectType)

In [33]:
dr_s = {'Type':doors_types,'Area':doors_areas}
doors_df = pd.DataFrame(dr_s)
doors_types_sum = doors_df.groupby('Type')['Area'].sum()

In [34]:
doors_types_sum_df =  pd.DataFrame(doors_types_sum)

In [35]:
doors_types_sum_df

,Area
Type,
M_Jednoskrzydłowe płytowe:0915 x 2134mm,3.189579


## Get the info about CW Mullions




In [36]:
cw_mullions = ifc_file.by_type('IfcMember')

In [37]:
quantities = ifcHelper.get_quantity_single_value(ifcHelper.get_related_quantities(cw_mullions[0])[0])

In [38]:
ifcHelper.get_related_quantities(cw_mullions[0])

[#9941=IfcElementQuantity('1TUusPmpLFzPBFUW$d6FcZ',#41,'BaseQuantities','',$,(#9939,#9940))]

In [39]:
quantities

{'Length': 21009.8295796457, 'CrossSectionArea': 168.078636637166}

In [40]:
cw_mullions_types = list()
cw_mullions_lengths = list()
for i in range(len(cw_mullions)):
    quantities_cw_mullions = ifcHelper.get_quantity_single_value(ifcHelper.get_related_quantities(cw_mullions[i])[0])
    cw_mullions_lengths.append(quantities_cw_mullions['Length']/1000)
    cw_mullions_types.append(cw_mullions[i].ObjectType)

In [41]:
cw_m_s = {'Type':cw_mullions_types,'Length':cw_mullions_lengths}
cw_mullions_df = pd.DataFrame(cw_m_s)
cw_mullions_types_sum = cw_mullions_df.groupby('Type')['Length'].sum()

In [42]:
cw_mullions_types_sum_df =  pd.DataFrame(cw_mullions_types_sum)

In [43]:
cw_mullions_types_sum_df

,Length
Type,
Prostokątny szpros:50 x 150mm,1302.609434


## Get the info about CW Panels



In [44]:
cw_panels = ifc_file.by_type('IfcPlate')

In [45]:
cw_panels[0]

#2898=IfcPlate('38uZGFQD9EDA60Kb6EDOKS',#41,'Panel systemowy:Bryła:332052',$,'Panel systemowy:Bryła',#2896,#2883,'332052')

In [46]:
cw_panels_types = list()
cw_panels_areas = list()
for i in range(len(cw_panels)):
    cw_panels_areas.append(ifcHelper.get_property_single_value(ifcHelper.get_related_property_sets(cw_panels[i])[1])['Area'])
    cw_panels_types.append(cw_panels[i].ObjectType)

In [47]:
cw_p_s = {'Type':cw_panels_types,'Area':cw_panels_areas}
cw_panels_df = pd.DataFrame(cw_p_s)
cw_panels_types_sum = cw_panels_df.groupby('Type')['Area'].sum()

In [48]:
cw_panels_types_sum

Type
Panel systemowy:Bryła           39.472594
Panel systemowy:Przeszklony    121.450327
Name: Area, dtype: float64

## Get the info about Railings




In [49]:
railings = ifc_file.by_type('IfcRailing')

In [59]:
railings_types = list()
railings_lengths = list()
for i in range(len(railings)):
    quantities_railings = ifcHelper.get_quantity_single_value(ifcHelper.get_related_quantities(railings[i])[0])
    railings_lengths.append(quantities_railings['Length'])
    railings_types.append(railings[i].ObjectType)

In [60]:
rl_s = {'Type':railings_types,'Length':railings_lengths}
railings_df = pd.DataFrame(rl_s)
railings_types_sum = railings_df.groupby('Type')['Length'].sum()

In [61]:
railings_types_sum

Type
Railing:1100mm                               19352.321841
Railing:Panel szklany - dolne wypełnienie     8296.436118
Name: Length, dtype: float64

## Summary


In [50]:
## DOORS
doors_types_sum_df

,Area
Type,
M_Jednoskrzydłowe płytowe:0915 x 2134mm,3.189579


In [51]:
## WINDOWS
windows_types_sum_df

,Area
Type,
M_Stałe:O01,0.776808
M_Stałe:O02,3.178754


In [52]:
## SLABS
slabs_types_sum_df

,Area
Type,
Basic Roof:Ogólne - 400 mm,1992.181745
Floor:Ogólne 150 mm,377.670000


In [53]:
## WALLS
wall_types_sum_df 

,Area
Type,
Basic Wall:Z01,323.497947
Basic Wall:Z02,229.437074


In [54]:
## CURTAIN WALL
cw_panels_types_sum

Type
Panel systemowy:Bryła           39.472594
Panel systemowy:Przeszklony    121.450327
Name: Area, dtype: float64

In [55]:
cw_mullions_types_sum_df

,Length
Type,
Prostokątny szpros:50 x 150mm,1302.609434
